# ***Child Mind Institute - Relating Physical Activity to Problematic Internet Use***
Link to GitHub for this project 👉 check it out

Here is an article from the institute on Summer Screen Time Use 👉 [check it out](https://childmind.org/article/screen-time-and-summer/)



### The Problem at Hand 🧑‍💻 *(taken from the comptetition homepage 👉 [check it out](https://www.kaggle.com/competitions/child-mind-institute-problematic-internet-use/overview))*

"In today’s digital age, problematic internet use among children and adolescents is a growing concern. Better understanding this issue is crucial for addressing mental health problems such as depression and anxiety.

Current methods for measuring problematic internet use in children and adolescents are often complex and require professional assessments. This creates access, cultural, and linguistic barriers for many families. Due to these limitations, problematic internet use is often not measured directly, but is instead associated with issues such as depression and anxiety in youth.

Conversely, physical & fitness measures are extremely accessible and widely available with minimal intervention or clinical expertise. Changes in physical habits, such as poorer posture, irregular diet, and reduced physical activity, are common in excessive technology users. We propose using these easily obtainable physical fitness indicators as proxies for identifying problematic internet use, especially in contexts lacking clinical expertise or suitable assessment tools."

**What does this mean?** The Child Mind Institute has tasked the public with building predictive machine learning models that will determine the Severity Impairment Indes (SII), measuring the level of problematic internet use among children and adolescents, based on physical activity, health, and lifestyle factors. The aim is to identify signs of problematic internet use early so that preventative measures can be taken by the parent/ caretaker.

### The Data at Hand 📊
In the institute's Healthy Brain Network (HBN) study, roughly 5,000 participants (aged 5-22) were given an accelerometer to wear for up to 30 days continually while at home and going about their daily lives. This data, known as **actigraphy data**, was collected alongside data from various records and assessments, including a Physical Activity Questionaire (PAQ), Parent-Child Internet Addiction Test (PCIAT), the FitnessGram Child Test, and others - this we'll call **feature data**.

Our objective is to train an ensemble of machine learning models on both of these datasets that will accurately predict the SII, the **target variable**, for each participant in what is known as the **test set**, a small dataset that is missing some of the features in the other sets including the SII.

We will do this by first **loading the data, cleaning the data (removing missing values), and preforming some simple EDA (exploratory data analysis),** to help us better understand what we're working with and **how different features relate to one another and the target variable**. Then we will engage in pre-processing and feature engineering to create a more robust dataset, and finally train and evaluate our models.

Because the natures of the features data and the actigraphy data are vastly different, we will need to analyze, process and engineer the datasets separately. The actigraphy data is a ***time series dataset**, which means we will get a lot of value from training a neural network on it. For the features data, simpler baseline ML models should be appropriate. Afterwards, we'll merge our results for final submission.

### Let's start by taking a peek into our data:

In [3]:
# load pandas
import pandas as pd

# load train set
train = pd.read_csv("/Users/tomragus/Library/CloudStorage/OneDrive-UCSanDiego/CMI-PIU-Model/data/train.csv")

# display first 5 rows of train set
print("""Train set: where the 'features' live""")
print(f"Train shape: {train.shape}")
display(train.head())

Train set: where the 'features' live
Train shape: (3960, 82)


,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# load test set
test = pd.read_csv("/Users/tomragus/Library/CloudStorage/OneDrive-UCSanDiego/CMI-PIU-Model/data/test.csv")

# display first 5 rows of test set
print("""Test set: what we will evaluate our models on""")
print(f"Test shape: {test.shape}")
display(test.head())

Test set: what we will evaluate our models on
Test shape: (20, 59)


,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,32.6909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,3.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,27.0552,NaN,NaN,Fall,2.340,Fall,46.0,64.0,Summer,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,45.9966,NaN,NaN,Winter,2.451,Summer,31.0,45.0,Winter,0.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,Summer,1.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


It can be tricky to figure out what all of these abbreviations mean - thankfully, the Child Mind Institute was kind enough to include a **data dictionary** for this competition, which gives some extra information for each variable. Here is a little preview - [you can view the full file on the Kaggle page](https://www.kaggle.com/competitions/child-mind-institute-problematic-internet-use/data?select=data_dictionary.csv).

In [6]:
# load data dictionary
data_dict = pd.read_csv("/Users/tomragus/Library/CloudStorage/OneDrive-UCSanDiego/CMI-PIU-Model/data/data_dictionary.csv")

# display first 5 rows of data dictionary
print("""Data Dictionary: what each feature means""")
print(f"Data Dictionary shape: {data_dict.shape}")
display(data_dict.head())

Data Dictionary: what each feature means
Data Dictionary shape: (81, 6)


,Instrument,Field,Description,Type,Values,Value Labels
0,Identifier,id,Participant's ID,str,NaN,NaN
1,Demographics,Basic_Demos-Enroll_Season,Season of enrollment,str,"Spring, Summer, Fall, Winter",NaN
2,Demographics,Basic_Demos-Age,Age of participant,float,NaN,NaN
3,Demographics,Basic_Demos-Sex,Sex of participant,categorical int,"0,1","0=Male, 1=Female"
4,Children's Global Assessment Scale,CGAS-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN


While this only gives us a snippet of the data at hand, we can see the SII on the right side of the train set as a rating ranging from 0 to 2 - though a closer investigation of the train set would reveal that the scores actually range from 0 to 3, with 0 representing no impairment, and 3 representing severe impairment. So essentially, what we want our models to do is **classify each id** into one of the 4 SII classes. 

Comparing the train and test sets, we can see that the train set has **82** variables, while the test set has **59**. They share some features, including id, but are missing a handful, including SII. The train set is also massive, with almost 4,000 entries. This is good - the more data we have, the more finely we can tune our models.

You might be thinking, but what about the actigraphy data? *The actigraphy data is trickier to look at,* since it's not neatly stored in a CSV file like the other data is - but that's not an issue. We'll get a sense of what it looks like in the EDA phase.

### Loading the rest of our libraries:

In [ ]:
# load all libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from datetime import datetime
import xgboost as xgb
import lightgbm as lgb
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

from matplotlib.colors import ListedColormap

### Exploratory Data Anlaysis

